In [1]:
!pip install transformers 
!pip install datasets 
!pip install evaluate 
!pip install trl 
!pip install huggingface_hub 
!pip install accelerate 
!pip install wandb 
!pip install scikit-learn
!pip install gpustat
!pip install --force-reinstall typing_extensions


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 361.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.7/792.7 kB 267.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 458.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 156.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 MB 255.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 430.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is avail

In [ ]:
from huggingface_hub import login
api_token = "Your Token id"
login(api_token)

In [1]:
import torch
import wandb
from sklearn.model_selection import train_test_split
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    pipeline, 
    Trainer
)
from transformers.integrations import WandbCallback
from trl import DataCollatorForCompletionOnlyLM
import evaluate
import datasets

model_name = "meta-llama/Llama-3.2-1B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_name,
    use_cache=False,
    device_map="auto",
    torch_dtype=torch.bfloat16, 
    low_cpu_mem_usage=True,
    attn_implementation="eager",
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token_id = tokenizer.eos_token_id

### 3.4.3. 데이터셋 

In [2]:
input_text = """부산의 한 왕복 2차선 도로에서 역주행 사고로 배달 오토바이 운전자인 고등학생이 숨지는 사고가 발생했다.
유족은 '가해자가 사고 후 곧바로 신고하지 않고 늑장 대응해 피해를 키웠다'고 주장하고 있다.
11일 부산진경찰서는 교통사고처리특례법(교통사고처리법)상 업무상 과실치사 혐의로 지난 3일 A(59)씨를 검찰에 불구속 송치했다고 밝혔다.
A씨는 교통사고처리법상 12대 중과실에 해당되는 '중앙선 침범'으로 역주행 교통사고를 일으킨 혐의를 받는다.
경찰에 따르면 스포츠유틸리티차량(SUV) 운전자 A씨는 5월 19일 밤 11시 50분쯤 부산진구 가야고가교 밑 도로에서 중앙선을 넘어 역주행으로 140m를 달려
반대편 차선의 오토바이 운전자 조모(16)군을 들이받았다. 조군은 원동기장치자전거 면허를 취득한 상태였고 헬멧도 쓰고 있었지만 크게 다쳤다.
사고 당일 수술을 받았으나 얼마 후 2차 뇌출혈로 뇌사 판정이 내려졌고, 사고 발생 약 한 달 만인 지난달 16일 끝내 사망했다.
사고를 낸 A씨는 술을 마시거나 약물을 복용한 상태에서 운전하지는 않은 것으로 조사됐다.
경찰 관계자는 'A씨가 자신이 정주행을 하고 오토바이가 역주행을 한 것으로 착각했다고 진술했다'고 설명했다."""

def change_inference_chat_format(input_text):
    return [
    {"role": "user", "content": f"다음 텍스트를 한국어로 간단히 요약 및 관련 키워드를 추출해주세요:\n{input_text}"},
	{"role": "assistant", "content": ""},
    ]
prompt = change_inference_chat_format(input_text)
# tokenizer 초기화 및 적용t\
inputs = tokenizer.apply_chat_template(prompt, tokenize=True, add_generation_prompt=True, return_tensors="pt").to(model.device)
outputs = model.generate(inputs, max_new_tokens=5102, use_cache=True)
print(tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 06 Jan 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

다음 텍스트를 한국어로 간단히 요약 및 관련 키워드를 추출해주세요:
부산의 한 왕복 2차선 도로에서 역주행 사고로 배달 오토바이 운전자인 고등학생이 숨지는 사고가 발생했다.
유족은 '가해자가 사고 후 곧바로 신고하지 않고 늑장 대응해 피해를 키웠다'고 주장하고 있다.
11일 부산진경찰서는 교통사고처리특례법(교통사고처리법)상 업무상 과실치사 혐의로 지난 3일 A(59)씨를 검찰에 불구속 송치했다고 밝혔다.
A씨는 교통사고처리법상 12대 중과실에 해당되는 '중앙선 침범'으로 역주행 교통사고를 일으킨 혐의를 받는다.
경찰에 따르면 스포츠유틸리티차량(SUV) 운전자 A씨는 5월 19일 밤 11시 50분쯤 부산진구 가야고가교 밑 도로에서 중앙선을 넘어 역주행으로 140m를 달려
반대편 차선의 오토바이 운전자 조모(16)군을 들이받았다. 조군은 원동기장치자전거 면허를 취득한 상태였고 헬멧도 쓰고 있었지만 크게 다쳤다.
사고 당일 수술을 받았으나 얼마 후 2차 뇌출혈로 뇌사 판정이 내려졌고, 사고 발생 약 한 달 만인 지난달 16일 끝내 사망했다.
사고를 낸 A씨는 술을 마시거나 약물을 복용한 상태에서 운전하지는 않은 것으로 조사됐다.
경찰 관계자는 'A씨가 자신이 정주행을 하고 오토바이가 역주행을 한 것으로 착각했다고 진술했다'고 설명했다.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

<|eot_id|><|start_header_id|>assistant<|end_header_id|>

다음은 요약과 관련 키워드의 한국어 번역:

 요약:
 - 부산의 왕복 2차선 도로에서 역주행 사고로 16세의 운전자 A(5

### 3.4.6 데이터 전처리

In [3]:
def chat_keyword_summary_format(example):

        return [
            {"role": "user", "content": f"다음 텍스트를 한국어로 간단히 요약 및 관련 키워드를 추출해주세요:\n{example['original']}"},
            {"role": "assistant", "content": f"한국어 요약:\n{example['summary']}\n키워드:\n{example['keywords']}"}
        ]

def tokenize(element):
    formatted = tokenizer.apply_chat_template(
        chat_keyword_summary_format(element), tokenize=False)
    outputs = tokenizer(formatted)
    return {
        "input_ids": outputs["input_ids"],
        "attention_mask": outputs["attention_mask"],
    }

sample_dataset = datasets.load_dataset("daje/keyword_summary")
print(sample_dataset["train"][0])
tokenized_sample_dataset = sample_dataset.map(tokenize)
tokenized_sample_dataset = tokenized_sample_dataset['train'].train_test_split(test_size=0.1, seed=42)
tokenized_sample_dataset


{'original': '전두환 정권을 흔들었던 \'큰 손\' 장영자의 네 번째 유죄가 확정됐다.\n  혐의는 사기, 징역은 4년이다.\n  대법원 2부(주심 박상옥 대법관)는 9일 무죄를 주장한 장씨의 상고를 기각했다.\n  장씨는 지인들에게서 세 차례에 걸쳐 6억여원을 가로채고 이들에게 위조 수표로 수억원을 요구한 혐의를 받는다.\n  올해로 76세를 맞은 장씨는 이미 복역을 마친 29년을 포함해 인생의 절반가량인 33년을 감옥에서 보내게됐다.\n   우수한 인재에게만 허락된다는 숙명여대 \'5월의 여왕(메이퀸)\' 출신인 장영자의 노년이 씁쓸하다.\n  장씨는 전두환 전 대통령의 처삼촌인 고(故) 이규광씨의 처제이자, 중앙정보부 차장이었던 고(故) 이철희씨의 아내였다.\n  전두환 정권 흔든 장영자의 삶장씨의 앞선 범행과 비교해 네 번째 사기 액수는 그 규모가 상당히 줄어든 편이다.\n  장씨는 1980년대 전두환 전 대통령과 남편을 내세워 자금 압박에 시달리던 기업에 자금을 빌려준 뒤 몇 배에 달하는 어음을 할인 유통하며 이득을 챙겼다.\n  그 규모가 총 7111억원에 달했다.\n  장씨는 이중 6404억원의 어음을 할인해 사용했다.\n   실체가 없던 어음은 부도가 났고 기업들은 도산했다.\n  장씨는 이 첫 번째 사기로 15년형을 선고받았다.\n  10년만인 1992년에 가석방됐다.\n  전두환 전 대통령의 부인 이순자씨는 자서전 『당신은 외롭지 않다』에서 이를 "조금씩 민심도 안정되고 경제도 생기를 되찾아 (남편이) 자신감을 얻던 시점에 날벼락같이 찾아온 횡액과도 같은 사건"이라 회고했다.\n  장씨가 당시 법정에서 아직 시중에 유통중인 어음이 있다며 "경제는 유통""나는 권력투쟁의 희생양이라"이라 했던 말은 유행어가 됐다.\n  \'장영자 어음할인 사건\' 수사 축소 의혹으로 두 명의 법무부 장관이 경질됐다.\n   "누나 성질 급하다" 70대 장영자의 돈 요구장씨는 출소한지 2년만인 1994년에 140억원대 차용 사기를 저질렀다.\n  두 번

DatasetDict({
    train: Dataset({
        features: ['original', 'summary', 'keywords', 'input_ids', 'attention_mask'],
        num_rows: 900
    })
    test: Dataset({
        features: ['original', 'summary', 'keywords', 'input_ids', 'attention_mask'],
        num_rows: 100
    })
})

### 3.4.7. 데이터셋 분리 및 콜레이터 설정

In [4]:
tokenizer.decode(tokenized_sample_dataset['train'][0]['input_ids'])

'<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 06 Jan 2025\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n다음 텍스트를 한국어로 간단히 요약 및 관련 키워드를 추출해주세요:\n\'동묘 길고양이 학대\' 사건을 계기로 동물 학대에 대한 법적 처벌을 강화해야 한다는 지적이 나온다.\n  지난 12일 유명 고양이 카페(고양이라서 다행이야)에는 서울 동대문구 동묘시장의 한 가게 앞에서 긴 쇠꼬챙이로 제압당하는 고양이 사진이 올라왔다.\n  글쓴이는 “상인 여러 명이 길고양이를 줄에 묶어 집어 던지고 목을 졸랐다”고 설명했다.\n  동물보호단체 회원들은 상인회 측에 항의 전화를 하고 이날 경찰에 고발 조치했다.\n 고발장을 접수한 경찰은 동묘시장 고양이 학대 사건과 관련해 당일 수사에 착수했다.\n  서울 혜화경찰서는 같은 날 상인 A씨를 동물보호법 위반 혐의로 입건했다고 16일 밝혔다.\n  경찰 관계자는 "CCTV는 물론, 목격자와 행인들을 모두 조사하고 있다"고 말했다.\n  상인 측은 가게에 들어온 고양이를 쫓아내기 위해 목줄을 묶어 밖으로 끌고 나간 것이라고 주장하며 혐의를 부인했다.\n  길고양이는 동물보호단체 ‘카라’가 구조해 현재 서울시 동물보호과 연계병원으로 옮겨졌다.\n  동물보호단체 측에 따르면 해당 고양이는 병원 이송 과정에서 침을 흘리는 등 쇼크 상태를 보였다.\n  복부 부위엔 다수의 찰과상을 입었다.\n   이 사건뿐만이 아니다.\n  지난 16일에도 서울 관악구 내 복지시설과 주차장 등지에선 처참하게 훼손된 고양이 사체가 여러 구 발견돼 경찰이 수사에 나섰다.\n  경찰에 따르면 지난달 22일 오전 4시 30분쯤 관악구 난곡동 복지관 인근에서 복부가 훼손된 고양이 사체가 발견됐다.\n  고양

### 3.4.8 학습 파라미터 설정 

In [5]:
#!wandb login
# 터미널에 wandb login --relogin

In [6]:
wandb.init(project="finetuning_view",
           #entity="wooseoksong",
           settings=wandb.Settings(init_timeout=120)
          )


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: wooseoksong (wooseoksong-study). Use `wandb login --relogin` to force relogin


### 3.4.9 평가 메트릭 정의 

In [7]:
bleu = evaluate.load("bleu")
acc = evaluate.load("accuracy")

def preprocess_logits_for_metrics(logits, labels):
    if isinstance(logits, tuple):
        # 모델과 설정에 따라 logits에는 추가적인 텐서들이 포함될 수 있습니다.
        # 예를 들어, past_key_values 같은 것들이 있을 수 있지만,
        # logits는 항상 첫 번째 요소입니다.
        logits = logits[0]
    # 토큰 ID를 얻기 위해 argmax를 수행합니다.
    return logits.argmax(dim=-1)

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # preds는 labels와 같은 형태를 갖습니다.
    # preprocess_logits_for_metrics에서 argmax(-1)가 계산된 후입니다.
    # 하지만 우리는 labels를 한 칸 이동시켜야 합니다.
    labels = labels[:, 1:]
    preds = preds[:, :-1]

    # -100은 DataCollatorForCompletionOnlyLM에서 사용되는 
    # ignore_index의 기본값입니다.
    mask = labels == -100
    # -100을 토크나이저가 디코드할 수 있는 값으로 대체합니다.
    labels[mask] = tokenizer.pad_token_id
    preds[mask] = tokenizer.pad_token_id

    # BLEU 점수는 텍스트를 입력으로 받기 때문에,
    # 토큰 ID에서 텍스트로 변환해야 합니다.
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    bleu_score = bleu.compute(predictions=decoded_preds, references=decoded_labels)
    
    # accuracy는 정수 리스트를 입력으로 받습니다.
    # 우리는 -100이 아닌 부분만 평가하고 싶기 때문에,
    # 마스크의 부정(~)을 사용합니다.
    accuracy = acc.compute(predictions=preds[~mask], references=labels[~mask])

    return {**bleu_score, **accuracy}

### 3.4.10 모델 학습 및 평가

In [8]:
training_args = TrainingArguments(
    output_dir="llama_results",
    run_name="llama_f",
    num_train_epochs=10 ,
    #max_steps=800,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    warmup_steps=0,
    weight_decay=0.01,
    learning_rate=2e-4,
    logging_dir="logs",
    evaluation_strategy="steps",
    logging_steps=100,
    save_steps = 1000,
    report_to="wandb",
    )

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [9]:
response_template_ids = tokenizer.encode(
    "assistant<|end_header_id|>", 
    add_special_tokens=False
    )
collator = DataCollatorForCompletionOnlyLM(
    response_template_ids, tokenizer=tokenizer
)
train_dataset_sampled = tokenized_sample_dataset["train"].select(range(500))
trainer = Trainer(
    args=training_args,
    model=model,
    tokenizer=tokenizer,
    data_collator=collator,
    train_dataset=train_dataset_sampled,
    eval_dataset=tokenized_sample_dataset["test"],
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    compute_metrics=compute_metrics,
    callbacks=[WandbCallback()]
)

/tmp/ipykernel_1016/2435741338.py:9: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
You are adding a <class 'transformers.integrations.integration_utils.WandbCallback'> to the callbacks of this Trainer, but there is already one. The currentlist of callbacks is
:DefaultFlowCallback
WandbCallback


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Bleu,Precisions,Brevity Penalty,Length Ratio,Translation Length,Reference Length,Accuracy
100,2.581700,1.975435,0.287666,"[0.6027691107644306, 0.4007557677008751, 0.25304383116883117, 0.17191383595691798]",0.898472,0.903294,5128,5677,0.726610
200,1.804800,1.738652,0.341811,"[0.6294106779974137, 0.435159043854696, 0.29081143295607137, 0.20829258752200275]",0.952399,0.953497,5413,5677,0.756531
300,1.344900,1.627088,0.363436,"[0.6253463883244043, 0.44701675136457747, 0.3184346825244581, 0.23821631136319185]",0.952399,0.953497,5413,5677,0.757693
400,1.241500,1.574413,0.381035,"[0.6434957124612297, 0.4608808771603791, 0.3291043362999432, 0.24917969503956766]",0.964872,0.965475,5481,5677,0.773661
500,1.212400,1.498674,0.423535,"[0.6612436731742588, 0.49300441826215025, 0.372655663915979, 0.294151376146789]",0.974129,0.974458,5532,5677,0.786296
600,0.853000,1.499557,0.412977,"[0.6680703039179787, 0.49477806788511747, 0.3639300646142151, 0.28302983339790777]",0.961402,0.962128,5462,5677,0.785986
700,0.841200,1.512436,0.400655,"[0.6631055442991343, 0.4860198911615688, 0.3534136546184739, 0.2715929031000195]",0.955347,0.956315,5429,5677,0.788078
800,0.682600,1.518140,0.430329,"[0.6738932410275096, 0.5060308034885879, 0.3811684628474192, 0.30256311428020816]",0.966330,0.966884,5489,5677,0.792574
900,0.496700,1.535433,0.443975,"[0.6835973056617514, 0.5184498423882811, 0.3961836387681844, 0.31638744463701135]",0.967058,0.967589,5493,5677,0.797458
1000,0.519500,1.479133,0.459544,"[0.6837284482758621, 0.5234089246525238, 0.4068554396423249, 0.3312452543659833]",0.980614,0.980800,5568,5677,0.804744


In [ ]:
trainer.evaluate()

#### 3.4.11 파인튜닝한 모델 테스트

In [ ]:
input_text = '''경찰이 6일 고위공직자범죄수사처(공수처)가 윤석열 대통령 체포영장의 집행 관련 업무를 경찰에 일임하겠다면서 발송한 공문에 법률적 논란이 있다면서 사실상 집행 거부 의사를 밝혔다.

백동흠 국가수사본부 비상계엄 특별수사단 부단장은 이날 브리핑에서 "내부적 법률 검토를 거쳐 공수처 집행 지휘 공문은 법률적 논란이 있는 것으로 판단했다"며 "체포영장 집행에 대해서는 공수처와 계속 협의해나가겠다"고 말했다.

백 부단장은 "공조수사본부 체제에서 법과 원칙에 따라 엄정하게 수사하고 있다"며 "앞으로도 체포영장 집행을 포함해 비상계엄 관련 수사를 위해 최선을 다하겠다"고 강조했다.

특별수사단은 이날 오전 7시께 공수처로부터 '체포영장 및 수색영장 집행지휘' 공문을 접수했다고 밝혔다.'''

def change_inference_chat_format(input_text):
    return [
    {"role": "user", "content": f"다음 텍스트를 한국어로 간단히 요약 및 관련 키워드를 추출해주세요:\n{input_text}"},
    {"role": "assistant", "content": ""}
    ]
prompt = change_inference_chat_format(input_text)
# tokenizer 초기화 및 적용t\
inputs = tokenizer.apply_chat_template(prompt, tokenize=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
outputs = model.generate(inputs, max_new_tokens=1024, use_cache=True)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))